In [0]:
%pip install featexp

In [0]:
import os
import kagglehub
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, classification_report
from pyspark.sql.functions import datediff, col
from pyspark.sql.functions import col, explode_outer, lpad, regexp_replace
from pyspark.sql import DataFrame, SparkSession
from pyspark.sql.functions import posexplode, concat, lit
import seaborn as sns


# Display all columns
pd.set_option('display.max_columns', None)

In [0]:
# read into a dataframe
raw=spark.sql(""" select diagnosis.claim_id,diagnosis.patient_id,diagnosis.date_service,diagnosis.date_service_end,diagnosis_code,enrollment.patient_gender,patient_year_of_birth,sum(procedure.line_charge) as paid_amt 
from healthverity_claims_sample_patient_dataset.hv_claims_sample.diagnosis 
inner join healthverity_claims_sample_patient_dataset.hv_claims_sample.enrollment on diagnosis.patient_id = enrollment.patient_id
inner join healthverity_claims_sample_patient_dataset.hv_claims_sample.procedure on diagnosis.patient_id = procedure.patient_id
where admit_diagnosis_ind='Y'
group by all""")
raw = raw.withColumnRenamed("date_service", "admission_dt").withColumnRenamed("date_service_end", "discharge_dt").withColumnRenamed("diagnosis_code","diag_cd")
display(raw)

In [0]:
# Example: Create age feature
df = raw.toPandas()
df['admission_dt'] = pd.to_datetime(df['admission_dt'])
df['discharge_dt'] = pd.to_datetime(df['discharge_dt']) 

# Feature engineering: use primary diagnosis/procedure codes, gender, LOS, etc.
df['length_of_stay'] = (df['discharge_dt'] - df['admission_dt']).dt.days.astype('int') 
print("Number of null length_of_stay values:", df['length_of_stay'].isnull().sum())

In [0]:
from featexp import get_univariate_plots

df_train = df
# Call the function to generate univariate plots
get_univariate_plots(df, 
                     'paid_amt', 
                     ['length_of_stay'])